# Install the prerequisites

In [ ]:
! pip install datasets
! pip install huggingface_hub
! huggingface-cli login --token YOUR_HF_TOKEN
! pip install -U sentence-transformers


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

In [ ]:
import json
# Function to process 'unstructured'
def process_unstructured(content):
    try:
        parsed = json.loads(content)
        return " ".join([k.get("content", k.get("text")) for k in parsed.values()])
    except json.JSONDecodeError:
        return content

# Function to process 'list_dump'
def process_list_dump(content):
    try:
        parsed = json.loads(content)
        return ' '.join(parsed)
    except json.JSONDecodeError:
        return content

# Function to process 'plain_text'
def process_plain_text(content):
    return content

#Apply functions conditionally based on 'content_type'
def process_content(row):
    if row['content_type'] == 'unstructured':
        return process_unstructured(row['content'])
    elif row['content_type'] == 'list_dump':
        return process_list_dump(row['content'])
    elif row['content_type'] == 'plain_text':
        return process_plain_text(row['content'])
    else:
        return row['content']  # Default case if needed

# Load data

In [ ]:
dataset = load_dataset("esg-x/esg-filtered-dataset", token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/483 [00:00<?, ?B/s]

train-00000-of-00009.parquet:   0%|          | 0.00/55.3M [00:00<?, ?B/s]

train-00001-of-00009.parquet:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

train-00002-of-00009.parquet:   0%|          | 0.00/409M [00:00<?, ?B/s]

...

train-00005-of-00009.parquet:   0%|          | 0.00/99.0M [00:00<?, ?B/s]

train-00006-of-00009.parquet:   0%|          | 0.00/101M [00:00<?, ?B/s]

train-00007-of-00009.parquet:   0%|          | 0.00/109M [00:00<?, ?B/s]

train-00008-of-00009.parquet:   0%|          | 0.00/118M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11134 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df['content_processed'] = df.apply(process_content, axis=1)

In [ ]:
df['text_length'] = df['content_processed'].apply(len)

# DeCLUTR split

## The copied from repo necessary function


In [ ]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device = "cuda")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.9 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
import torch
import pandas as pd


def split_in_chunks(text, tokenizer, chunksize=384):
    """
    Function to split the text in 512 chunks
    :return: the chunks
    """
    # tokenize
    tokens = tokenizer.encode_plus(text, add_special_tokens=False, return_tensors='pt')
    # chunk input ids and mask into chunksize-2 to leave space for 2 tokens for beginning and end: [101], [102]
    input_id_chunks = list(tokens['input_ids'][0].split(chunksize-2))
    mask_chunks = list(tokens['attention_mask'][0].split(chunksize-2))

    for i in range(len(input_id_chunks)):
        # append tokens for beginning of sentence
        input_id_chunks[i] = torch.cat([
            torch.Tensor([101]), input_id_chunks[i], torch.Tensor([102])
        ])
        mask_chunks[i] = torch.cat([
            torch.Tensor([1]), mask_chunks[i], torch.Tensor([1])
        ])

        pad_len = chunksize - input_id_chunks[i].shape[0]
        # adding the padding for "incomplete" sentence
        if pad_len > 0:
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.Tensor([0] * pad_len)
            ])
            mask_chunks[i] = torch.cat([
                mask_chunks[i], torch.Tensor([0] * pad_len)
            ])
    # put in the right format for the BERT model
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)

    input_dict = {
        'input_ids': input_ids.long(),
        'attention_mask': attention_mask.int()
    }
    return input_dict

In [ ]:
sentences = df["content_processed"][0]
encoded_input = split_in_chunks(sentences, tokenizer, chunksize=384)

Token indices sequence length is longer than the specified maximum sequence length for this model (10803 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
from typing import Callable, List, Optional, Tuple

import numpy as np

def sample_anchor_positive_pairs(
    text: str
) -> Tuple[List[str], List[str], List[str]]:
    """Returns a `Tuple` of `List`s, containing `num_anchors` anchor spans and `num_positives`
    positive spans sampled from `text`.

    # Parameters

    text : `str`, required
        The string to extract anchor and positive spans from.
    num_anchors : `int`, required
        The number of spans to sample from `text` to serve as anchors.
    num_positives : `int`, required
        The number of spans to sample from `text` to serve as positives (per anchor).
    max_span_len : `int`, required
        The maximum length of spans, after tokenization, to sample.
    min_span_len : `int`, required
        The minimum length of spans, after tokenization, to sample.
    sampling_strategy : `str`, optional (default = `None`)
        One of `"subsuming"` or `"adjacent"`. If `"subsuming"`, positive spans are always subsumed
        by the anchor. If `"adjacent"`, positive spans are always adjacent to the anchor. If not
        provided, positives may be subsumed, adjacent to, or overlapping with the anchor.
    tokenizer : `Callable`, optional (default = `None`)
        Optional tokenizer to use before sampling spans. If `None`, `text.split()` is used.
    """
    # Tokenize the incoming text. Whitespace tokenization is much more straightforward
    # (we don't need to worry about chopping up subword tokens), but a user can also provide
    # their own tokenization scheme if they want.
    num_anchors = 2
    num_positives = 2 # maybe change
    max_span_len = 256 # maybe 256
    min_span_len = 82 # maybe change, 82 is average number of charact in a sentence
    sampling_strategy =  "adjacent" #"adjacent" None
    tokenizer = None


    tokens = model.encode(text) if tokenizer is not None else text.split()
    tok_method = "tokenizer(text)" if tokenizer else "text.split()"
    num_tokens = len(tokens)

    if num_tokens < num_anchors * max_span_len * 2:
        raise ValueError(
            f"len({tok_method}) should be at least {num_anchors * max_span_len * 2}"
            f" (num_anchors * max_span_len * 2), got {num_tokens}."
        )
    if min_span_len > max_span_len:
        raise ValueError(
            f"min_span_len must be less than max_span_len ({max_span_len}), got {min_span_len}."
        )
    if max_span_len > num_tokens:
        raise ValueError(
            (
                f"max_span_len must be less than or equal to"
                f" len({tok_method}) ({num_tokens}), got {max_span_len}."
            )
        )

    # Valid anchor starts are token indices which begin a token span of at least max_span_len.
    anchors, positives = [], []
    anchor_positive_pairs = []
    valid_anchor_starts = list(range(0, num_tokens - max_span_len + 1, max_span_len))
    for i in range(num_anchors):
        # Sample the anchor length from a beta distribution skewed towards longer spans, the
        # intuition being that longer spans have the best chance of being representative of the
        # document they are sampled from.
        anchor_len = int(np.random.beta(4, 2) * (max_span_len - min_span_len) + min_span_len)
        # This check prevents an edge case were we run out of valid_anchor_starts.
        if len(valid_anchor_starts) // (num_anchors - i) < num_anchors - i:
            anchor_start_idx = np.random.choice([0, len(valid_anchor_starts) - 1])
        else:
            anchor_start_idx = np.random.randint(len(valid_anchor_starts))
        # When num_anchors = 1, this is equivalent to uniformly sampling that starting position.
        anchor_start = np.random.randint(
            valid_anchor_starts[anchor_start_idx],
            # randint is high-exclusive
            valid_anchor_starts[anchor_start_idx] + max_span_len - anchor_len + 1,
        )
        # Once sampled, remove an anchor (and its immediate neighbours) from consideration.
        del valid_anchor_starts[max(0, anchor_start_idx - 1) : anchor_start_idx + 2]
        anchor_end = anchor_start + anchor_len
        anchors.append(" ".join(tokens[anchor_start:anchor_end]))
        anchor = " ".join(tokens[anchor_start:anchor_end])
        # Sample positives from around the anchor. The intuition being that text that appears
        # close together is the same document is likely to be semantically similar.
        for _ in range(num_positives):
            # A user can specify a subsuming or adjacent only sampling strategy.
            if sampling_strategy == "subsuming":
                # To be strictly subsuming, we cannot allow the positive_len > anchor_len.
                positive_len = int(
                    np.random.beta(2, 4) * (anchor_len - min_span_len) + min_span_len
                )
                # randint is high-exclusive
                positive_start = np.random.randint(anchor_start, anchor_end - positive_len + 1)
            elif sampling_strategy == "adjacent":
                # Restrict positives to a length that will allow them to be adjacent to the anchor
                # without running off the edge of the document. If the anchor has sufficent room on
                # either side, this won't be a problem and max_positive_len will equal max_span_len.
                max_positive_len = min(max_span_len, max(anchor_start, num_tokens - anchor_end))
                if max_positive_len < max_span_len:
                    print(
                        (
                            "There is no room to sample an adjacent positive span. Temporarily"
                            " reducing the maximum span length of positives. This message will not"
                            " be displayed again."
                        )
                    )
                positive_len = int(
                    np.random.beta(2, 4) * (max_positive_len - min_span_len) + min_span_len
                )
                # There are two types of adjacent positives, those that border the beginning of the
                # anchor and those that border the end. The checks above guarantee at least one of
                # these is valid. Here we just choose from the valid positive starts at random.
                valid_starts = []
                if anchor_start - positive_len > 0:
                    valid_starts.append(anchor_start - positive_len)
                if anchor_end + positive_len <= num_tokens:
                    valid_starts.append(anchor_end)
                positive_start = np.random.choice(valid_starts)
            else:
                # Sample positive length from a beta distribution skewed towards shorter spans. The
                # idea is to promote diversity and minimize the amount of overlapping text.
                positive_len = int(
                    np.random.beta(2, 4) * (max_span_len - min_span_len) + min_span_len
                )
                # By default, spans may be adjacent or overlap with each other and the anchor.
                # Careful not to run off the edges of the document (this error may pass silently).
                positive_start = np.random.randint(
                    max(0, anchor_start - positive_len),
                    min(anchor_end, num_tokens - positive_len) + 1,  # randint is high-exclusive
                )

            positive_end = positive_start + positive_len
            positives.append(" ".join(tokens[positive_start:positive_end]))
            positive = " ".join(tokens[positive_start:positive_end])
            anchor_positive_pairs.append((anchor, positive))

    return anchors, positives, anchor_positive_pairs

## Apply

In [ ]:
df['num_tokens'] = df['content_processed'].apply(lambda text: len(text.split()))

In [ ]:
df = df[df['num_tokens'] > 1028]

In [ ]:
df.head()

,content,datatype,dataset,content_type,unique_id,__index_level_0__,content_processed,text_length,num_tokens
0,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,kaggle_reports,plain_text,0,0,Sustainability Highlight Report CARE BEYOND SK...,56786,8637
1,Corporate Responsibility Report 2021 2 Content...,sustainability_report,kaggle_reports,plain_text,1,1,Corporate Responsibility Report 2021 2 Content...,619607,94088
2,VONOVIA SE SUSTAINABILITY REPORT 2021 =For a S...,sustainability_report,kaggle_reports,plain_text,2,2,VONOVIA SE SUSTAINABILITY REPORT 2021 =For a S...,444517,68376
3,Sustainability Report 2021 TABLE OF CONTENTS S...,sustainability_report,kaggle_reports,plain_text,3,3,Sustainability Report 2021 TABLE OF CONTENTS S...,543061,80152
4,Our ideas and concepts FOR A SUSTAINABLE FUTUR...,sustainability_report,kaggle_reports,plain_text,4,4,Our ideas and concepts FOR A SUSTAINABLE FUTUR...,256566,38975


In [ ]:
#df.drop(columns=['anchors', 'positives'], inplace=True)

In [ ]:
df[['anchors', 'positives', 'anchor_positive']] = df['content_processed'].apply(
    sample_anchor_positive_pairs
).apply(pd.Series)

In [ ]:
df.head()

,content,datatype,dataset,content_type,unique_id,__index_level_0__,content_processed,text_length,num_tokens,anchors,positives,anchor_positive
0,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,kaggle_reports,plain_text,0,0,Sustainability Highlight Report CARE BEYOND SK...,56786,8637,[products. Find more information on the Consum...,[guarantee the highest quality and product saf...,[(products. Find more information on the Consu...
1,Corporate Responsibility Report 2021 2 Content...,sustainability_report,kaggle_reports,plain_text,1,1,Corporate Responsibility Report 2021 2 Content...,619607,94088,[less in extent than for a reasonable assuranc...,[perform the assurance engagement to allow us ...,[(less in extent than for a reasonable assuran...
2,VONOVIA SE SUSTAINABILITY REPORT 2021 =For a S...,sustainability_report,kaggle_reports,plain_text,2,2,VONOVIA SE SUSTAINABILITY REPORT 2021 =For a S...,444517,68376,[Governance and Material topics are defined by...,"[of its 2030 Agenda. The 17 main goals, which ...",[(Governance and Material topics are defined b...
3,Sustainability Report 2021 TABLE OF CONTENTS S...,sustainability_report,kaggle_reports,plain_text,3,3,Sustainability Report 2021 TABLE OF CONTENTS S...,543061,80152,[for renewable electricity to-date. We joined ...,"[heating, ventilation and air conditioning sys...",[(for renewable electricity to-date. We joined...
4,Our ideas and concepts FOR A SUSTAINABLE FUTUR...,sustainability_report,kaggle_reports,plain_text,4,4,Our ideas and concepts FOR A SUSTAINABLE FUTUR...,256566,38975,[to increase the proportion of women in manage...,[studying and in research • “Project U” projec...,[(to increase the proportion of women in manag...


In [ ]:
# Expand each list of anchor_positive pairs into separate rows
df_expanded = df.explode('anchor_positive')
# Then, split the pairs into 'anchor' and 'positive' columns
df_expanded[['anchor', 'positive']] = pd.DataFrame(df_expanded['anchor_positive'].tolist(), index=df_expanded.index)

In [ ]:
df_expanded["positive"].head(4).head(2).head(1)[0]

'guarantee the highest quality and product safety With 140 years of skin care expertise, it is a matter of course for us to develop skin care products that meet the highest quality standards and are absolutely safe to use. Our consumers expect this from us, and we do everything we can to meet their demands. We use scientific methods of the highest standards to extensively research our products and active ingredients to better understand their effect on skin and hair and to ensure the excellent quality and safety of our'

In [ ]:
df_expanded["anchor"].head(4).head(2).tail(1)[0]

'products. Find more information on the Consumer Business Segment here 11 CARE BEYOND SKIN OUR SUSTAINABILITY ENVIRONMENTAL FOOTPRINT OUR SOCIAL IMPACT COMMITMENT Our Targets Our Targets Protecting nature and the environment is the focus of our Sustainability Agen da CARE BEYOND SKIN. In the ENVI RONMENT area, we have defined four focus fields in which we can and want to achieve a great deal. We have set ourselves ambitious targets to minimize our environmental footprint progressively in all areas. TARGET 4 TARGET 3 TARGET 2 TARGET 1 SDG 30% absolute reduction in Scope 1, 2 and 3 greenhouse gas emissions by 2025 (vs. 2018 base year) PROGRESS Project ongoing Climate-neutral production sites by 2030 PROGRESS Project ongoing 50% reduction in petroleum-based virgin plastics in our packaging by 2025 (vs. 2019 base year) PROGRESS Project ongoing 30% recyclate in our plastic packaging by 2025 (vs. 2019 base year) PROGRESS Project ongoing 100% of our packaging refillable, reusable or recyclabl

In [ ]:
df_expanded.head()

,content,datatype,dataset,content_type,unique_id,__index_level_0__,content_processed,text_length,num_tokens,anchors,positives,anchor_positive,anchor,positive
0,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,kaggle_reports,plain_text,0,0,Sustainability Highlight Report CARE BEYOND SK...,56786,8637,[products. Find more information on the Consum...,[guarantee the highest quality and product saf...,(products. Find more information on the Consum...,products. Find more information on the Consume...,guarantee the highest quality and product safe...
0,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,kaggle_reports,plain_text,0,0,Sustainability Highlight Report CARE BEYOND SK...,56786,8637,[products. Find more information on the Consum...,[guarantee the highest quality and product saf...,(products. Find more information on the Consum...,products. Find more information on the Consume...,"ranges. Among other initiatives, we have imple..."
0,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,kaggle_reports,plain_text,0,0,Sustainability Highlight Report CARE BEYOND SK...,56786,8637,[products. Find more information on the Consum...,[guarantee the highest quality and product saf...,(further example with this new logistics hub. ...,further example with this new logistics hub. O...,women in Executive Board - 35% women in Manage...
0,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,kaggle_reports,plain_text,0,0,Sustainability Highlight Report CARE BEYOND SK...,56786,8637,[products. Find more information on the Consum...,[guarantee the highest quality and product saf...,(further example with this new logistics hub. ...,further example with this new logistics hub. O...,innovation and sustainability. Our new Leipzig...
1,Corporate Responsibility Report 2021 2 Content...,sustainability_report,kaggle_reports,plain_text,1,1,Corporate Responsibility Report 2021 2 Content...,619607,94088,[less in extent than for a reasonable assuranc...,[perform the assurance engagement to allow us ...,(less in extent than for a reasonable assuranc...,less in extent than for a reasonable assurance...,perform the assurance engagement to allow us t...


In [ ]:
# Drop the intermediate column
df_expanded = df_expanded.drop(columns=['anchor_positive','anchors', 'positives'])

In [ ]:
df_expanded = df_expanded.drop(columns=['__index_level_0__',	'content_processed',	'text_length',	'num_tokens'])
df_expanded = df_expanded.drop(columns=['content',	'datatype',	'dataset',	'content_type'])
df_expanded = df_expanded.rename(columns={'unique_id': 'doc_id'})
df_expanded['pairs_unique_id'] = range(0, len(df_expanded))

In [ ]:
df_expanded = df_expanded.reset_index(drop=True)

In [ ]:
df_expanded.head(10)

,doc_id,anchor,positive,pairs_unique_id
0,0,products. Find more information on the Consume...,guarantee the highest quality and product safe...,0
1,0,products. Find more information on the Consume...,"ranges. Among other initiatives, we have imple...",1
2,0,further example with this new logistics hub. O...,women in Executive Board - 35% women in Manage...,2
3,0,further example with this new logistics hub. O...,innovation and sustainability. Our new Leipzig...,3
4,1,less in extent than for a reasonable assurance...,perform the assurance engagement to allow us t...,4
5,1,less in extent than for a reasonable assurance...,We have performed a limited assurance engageme...,5
6,1,Willingness to recommend Deutsche Telekom as a...,all 111 of the national companies sur- entire ...,6
7,1,Willingness to recommend Deutsche Telekom as a...,compliance We receive complaints about possibl...,7
8,2,Governance and Material topics are defined by ...,"of its 2030 Agenda. The 17 main goals, which a...",8
9,2,Governance and Material topics are defined by ...,"of its 2030 Agenda. The 17 main goals, which a...",9


In [ ]:
len(df_expanded)

43476

In [ ]:
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43476 entries, 0 to 43475
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   doc_id           43476 non-null  int64 
 1   anchor           43476 non-null  object
 2   positive         43476 non-null  object
 3   pairs_unique_id  43476 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.3+ MB


# Push to HUB

In [ ]:
!git config --global credential.helper store

In [ ]:
from datasets import Dataset, Features, Value
# Define the schema (optional but recommended)
features = Features({
    'pairs_unique_id': Value('int64'),
    'doc_id': Value('int64'),
    'anchor': Value('string'),
    'positive': Value('string')
})

# Convert the DataFrame to a Dataset
expanded_dataset = Dataset.from_pandas(df_expanded, features=features)

In [ ]:
expanded_dataset

Dataset({
    features: ['pairs_unique_id', 'doc_id', 'anchor', 'positive'],
    num_rows: 43476
})

In [ ]:
df = pd.DataFrame(expanded_dataset)

In [ ]:
def create_negative_pairs(df):
    negative_pairs = []

    # Group by 'doc_id' for efficient negative sampling
    grouped = df.groupby('doc_id')

    for idx, row in df.iterrows():
        # Get current anchor and positive
        anchor = row['anchor']
        positive = row['positive']
        doc_id = row['doc_id']

        # Sample a negative example from a different 'doc_id'
        other_docs = df[df['doc_id'] != doc_id]
        if not other_docs.empty:
            negative_row = other_docs.sample(1).iloc[0]
            negative = negative_row['positive']

            # Append the anchor, positive, negative triplet
            negative_pairs.append({
                'pairs_unique_id': row['pairs_unique_id'],
                'doc_id': doc_id,
                'anchor': anchor,
                'positive': positive,
                'negative': negative
            })

    return pd.DataFrame(negative_pairs)

In [ ]:
triplet_df = create_negative_pairs(df)

In [ ]:
len(triplet_df)

43476

In [ ]:
hf_dataset = Dataset.from_pandas(triplet_df)
#hf_dataset = hf_dataset.remove_columns("pairs_unique_id")

In [ ]:
from datasets import DatasetDict
train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)

# Step 2: Further Split Train into Train and Validation (75% Train, 25% Validation from the 80% Train)
train_valid_split = train_test_split['train'].train_test_split(test_size=0.25, seed=42)

# Step 3: Combine splits into a DatasetDict
dataset = DatasetDict({
    'train': train_valid_split['train'],
    'val': train_valid_split['test'],  # This is the validation set
    'test': train_test_split['test']
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['pairs_unique_id', 'doc_id', 'anchor', 'positive', 'negative'],
        num_rows: 26085
    })
    val: Dataset({
        features: ['pairs_unique_id', 'doc_id', 'anchor', 'positive', 'negative'],
        num_rows: 8695
    })
    test: Dataset({
        features: ['pairs_unique_id', 'doc_id', 'anchor', 'positive', 'negative'],
        num_rows: 8696
    })
})

In [ ]:
dataset.push_to_hub("esg-x/esg-mostly-adjacent-dataset", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/esg-x/esg-mostly-adjacent-dataset/commit/098b2d9e1158716f79761f2db34be0f07b792c58', commit_message='Upload dataset', commit_description='', oid='098b2d9e1158716f79761f2db34be0f07b792c58', pr_url=None, pr_revision=None, pr_num=None)